<a href="https://colab.research.google.com/github/anasshaddad/DetectionColor/blob/main/DetectionColor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
kernel = np.ones((5, 5), np.uint8)
ksize = (4, 4) 


   
def applyMask(mask,hsv):#detect colors
    target = cv2.bitwise_and(hsv,hsv, mask=mask)
    target = cv2.GaussianBlur(target, (5, 5), 0)
    target = cv2.erode(target,kernel,5)
    edges = cv2.Canny(target,0,200)
    edges = cv2.dilate(edges, kernel) 
    return edges
def DefineSecteur(point,ImageHeight):
  SecteurField=ImageHeight/3  
  if(point<SecteurField):
      Secteur="Secteur 1"
  elif(point>SecteurField and point<2*SecteurField):
      Secteur ="Secteur 2"
  
  else:
      Secteur ="Secteur 3"
  return Secteur 
def DetectColor(edges,image,color,imageHeight):#draw color and centroid
    
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours :
    
       
      
         
        x,y,w,h = cv2.boundingRect(c)
        if(w*h>26000):
          cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
          point1=(x,y)
          point2=(x+w,y+h)
          cX= (point1[0]+point2[0])/2
          cY= (point1[1]+point2[1])/2
          part=DefineSecteur(cY,imageHeight)
        
          cv2.putText(image, "{},{}".format(color,part), (int(cX)-70, int(cY)),cv2.FONT_HERSHEY_SIMPLEX,0.7, (0, 0,0), 1)
   

cap = cv2.VideoCapture('C:/Users/manas/OneDrive/Bureau/paperEval.mp4')

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, image = cap.read()
  if ret == True:
    image1 = image
    image= cv2.erode(image,kernel)
    image = cv2.dilate(image, kernel) 

    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

    image = cv2.blur(image, ksize, cv2.BORDER_DEFAULT) 
    image =cv2.GaussianBlur(image,(5,5),0) 


## convert to hsv
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)


    colors={"mask1":[cv2.inRange(hsv, (0,100,0), (9, 255, 255)) ,"red"],
        "mask3":[cv2.inRange(hsv, (9,80,0), (21, 255, 255)),"yellow"],
        "mask4":[cv2.inRange(hsv, (23, 0, 0), (70, 255,255)),"green"],
        "mask5":[cv2.inRange(hsv, (70, 0, 0), (105, 255,255)),"cion"],
        "mask6":[cv2.inRange(hsv, (105,0,0), (125, 255, 255)),"Blue"],
        "mask7":[cv2.inRange(hsv, (128, 90, 125), (140, 255,255)),"magenta"],
        "mask8":[cv2.inRange(hsv, (170,130,0), (180, 255, 255)),"orange"],
        "mask9":[cv2.inRange(hsv, (140, 100, 100), (160, 255,255)),"pink"]
        }
    for masks in colors:
     mask = colors[masks][0]
     color = colors[masks][1]
     edge = applyMask(mask,hsv)
     DetectColor(edge,image1,color,frame_height)
    # Display the resulting frame
    out.write(image1)

    cv2.imshow('Frame',image1)

    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()


